# Statistical Analysis of the Lab Normality Paper

### Author: Song Xu

In [2]:
import stats_utils
from scripts.LabTestAnalysis.machine_learning import LabNormalityLearner_Legacy as LNL
from scripts.LabTestAnalysis.machine_learning.ml_utils import map_lab

import LocalEnv
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import metrics 

import warnings
warnings.filterwarnings("ignore")

stats_utils.py:111: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'agg' by the following code:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/ipykernel/kernelapp.py"

## Setting up variables

- Assign Data Set and Type here

In [3]:
data_source = 'Stanford'
lab_type = 'panel'
curr_version = '10000-episodes-lastnormal'
inverse01 = True # Setting 'True' to interpret 'Normal' as 'Negative'

In [4]:
project_folderpath = os.path.join(LocalEnv.PATH_TO_CDSS, 'scripts/LabTestAnalysis')
stats_folderpath = os.path.join(project_folderpath, 'lab_statistics')
ML_folderpath = os.path.join(project_folderpath, 'machine_learning')

In [5]:
inverse_maker = '_inversed01' if inverse01 else ''

dataSet_foldername = 'data-%s-%s-%s'%(data_source, 
                                      lab_type, 
                                      curr_version)
dataML_folderpath = os.path.join(ML_folderpath, dataSet_foldername)
dataStats_folderpath = os.path.join(stats_folderpath, dataSet_foldername)


In [6]:
all_labs = stats_utils.get_all_labs(data_source, lab_type)

labDescriptions = stats_utils.get_lab_descriptions(data_source=data_source, 
                                                   lab_type=lab_type)

## Figure 1: plot_full_cartoon of LABLDH

In [7]:
lab = 'LABLDH'
score_thres = 0.756
include_threshold_colors = True

figure1_folderpath = os.path.join(stats_folderpath, 'Fig1_cartoon')
if not os.path.exists(figure1_folderpath):
    os.mkdir(figure1_folderpath)

In [58]:
xVal_base, yVal_base, score_base, xVal_best, yVal_best, score_best, p_val \
            = stats_utils.get_curve_onelab(lab,
                                           all_algs=['random-forest'],
                                           data_folder=dataML_folderpath,
                                           curve_type='ROC',
                                           get_pval=False)

In [59]:
plt.figure(figsize=(5, 4))
# plt.plot(xVal_base, yVal_base, label='baseline model, %0.2f' % (score_base), linewidth=2)
'''Representative ROC of LABLDH'''
if not inverse01:
    plt.plot(xVal_best, yVal_best, color='orange', linewidth=2) #, label='random forest', AUROC=%0.2f  % (score_best)
else:
    plt.plot(1-yVal_best, 1-xVal_best, color='orange', linewidth=2)

if include_threshold_colors:
    df_directcompare_rf = pd.read_csv(os.path.join(dataML_folderpath, lab, 'random-forest', 'direct_comparisons.csv'))
    actual_labels = df_directcompare_rf['actual'].values
    predict_probas = df_directcompare_rf['predict'].values

    sensitivity, specificity, LR_p, LR_n, PPV, NPV = stats_utils.get_confusion_metrics(actual_labels, predict_probas, score_thres, also_return_cnts=False)
    print "sensitivity", sensitivity
    print "specificity", specificity
    print "score_thres", score_thres

    '''The POINT of PPV=0.95'''
    if not inverse01:
        plt.scatter(1-specificity, sensitivity, s=50, color='orange')
    else:
        plt.scatter(1-sensitivity, specificity, s=50, color='orange')

    '''Reference line of AUC=0.5'''
    dash_num = 20
    # plt.plot([1-specificity]*dash_num, np.linspace(0,1,num=dash_num), 'k--')
    plt.plot(np.linspace(0,1,num=dash_num),np.linspace(0,1,num=dash_num), color='lightblue', linestyle='--')

plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xticks([])
plt.yticks([])
plt.ylabel('Sensitivity', fontsize=16) #lab_descriptions.get(lab, lab)
plt.xlabel('1-Specificity', fontsize=16)
# plt.legend(fontsize=12)
plt.savefig(os.path.join(figure1_folderpath, 'ROC_%s%s.png'%(lab,inverse_maker)))

plt.clf()

df = pd.read_csv(dataML_folderpath + "/%s/baseline_comparisons.csv"
                 % (lab), keep_default_na=False)
scores_actual_0 = df.ix[df['actual'] == 0, 'predict'].values
scores_actual_1 = df.ix[df['actual'] == 1, 'predict'].values

plot_baseline = False
if plot_baseline:
    plt.figure(figsize=(5, 4))


    plt.hist(scores_actual_0, bins=30, alpha=0.8, color='b', label="Abnormal")
    plt.hist(scores_actual_1, bins=30, alpha=0.8, color='g', label="Normal")
    plt.xlim([0, 1])
    plt.ylim([0, 500])
    plt.xticks([])
    plt.yticks([])
    # plt.xlabel(lab_descriptions[lab] + 'auroc=%.2f' % auc)
    # plt.xlabel('baseline', fontsize=16)
    plt.xlabel('Score, baseline', fontsize=16)
    plt.ylabel('num of orders', fontsize=16)
    plt.legend(fontsize=12)
    plt.savefig(os.path.join(figure1_folderpath, 'cartoon_baseline_%s.png'%lab))
    plt.clf()

plt.figure(figsize=(5, 4))
alg = 'random-forest'
df = pd.read_csv(dataML_folderpath + "/%s/%s/direct_comparisons.csv"
                 % (lab, alg), keep_default_na=False)

df1 = pd.read_csv(dataML_folderpath + "/%s/%s/%s-normality-prediction-%s-report.tab"
                  % (lab, alg, lab, alg), sep='\t', keep_default_na=False)
auc = df1['roc_auc'].values[0]

if include_threshold_colors:
    scores_actual_trueNega = df.ix[(df['actual']==0) & (df['predict']<score_thres), 'predict'].values
    scores_actual_falsPosi = df.ix[(df['actual'] == 0) & (df['predict'] >= score_thres), 'predict'].values

    scores_actual_falsNega = df.ix[(df['actual'] == 1) & (df['predict'] < score_thres), 'predict'].values
    scores_actual_truePosi = df.ix[(df['actual'] == 1) & (df['predict'] >= score_thres), 'predict'].values

    if not inverse01:
        plt.hist(scores_actual_trueNega, bins=22, alpha=0.8, color='royalblue', label="true negatives")
        plt.hist(scores_actual_falsNega, bins=22, alpha=0.8, color='gold', label="false negatives")
        plt.hist(scores_actual_truePosi, bins=7, alpha=0.8, color='forestgreen', label="true positives")
        plt.hist(scores_actual_falsPosi, bins=7, alpha=0.8, color='orangered', label="false positives")

        plt.plot([score_thres] * dash_num, np.linspace(0, 800, num=dash_num), 'k--')
    else:
        plt.hist(1-scores_actual_trueNega, bins=22, alpha=0.8, color='royalblue', label="true positives")
        plt.hist(1-scores_actual_falsNega, bins=22, alpha=0.8, color='gold', label="false positives")
        plt.hist(1-scores_actual_truePosi, bins=7, alpha=0.8, color='forestgreen', label="true negatives")
        plt.hist(1-scores_actual_falsPosi, bins=7, alpha=0.8, color='orangered', label="false negatives")

        plt.plot([1-score_thres] * dash_num, np.linspace(0, 800, num=dash_num), 'k--')



    plt.legend(loc=(0.45,0.6), fontsize=12)

else:

    scores_actual_0 = df.ix[df['actual'] == 0, 'predict'].values
    scores_actual_1 = df.ix[df['actual'] == 1, 'predict'].values

    if not inverse01:
        plt.hist(scores_actual_0, bins=30, alpha=0.8, color='gray', label="Abnormal") #gray red
        plt.hist(scores_actual_1, bins=30, alpha=0.8, color='black', label="Normal") #black green
    else:
        plt.hist(1-scores_actual_0, bins=30, alpha=0.8, color='gray', label="Positive")
        plt.hist(1-scores_actual_1, bins=30, alpha=0.8, color='black', label="Negative")

    plt.legend(fontsize=12)

plt.xlim([0, 1])
plt.ylim([0, 800])
plt.xticks([])
plt.yticks([])
# plt.xlabel(lab_descriptions[lab])
# plt.xlabel('random forest', fontsize=16)
plt.xlabel('Score', fontsize=16)
plt.ylabel('Number of orders', fontsize=16)

if include_threshold_colors:
    plt.savefig(os.path.join(figure1_folderpath, 'cartoon_%s_thres%s.png' % (lab, inverse_maker)))
else:
    plt.savefig(os.path.join(figure1_folderpath, 'cartoon_%s%s.png' % (lab, inverse_maker)))

sensitivity 0.636603028308
specificity 0.96447467876
score_thres 0.756


## Component Transfer Stats

- Setting the config

In [12]:
def statistic_analysis(lab, dataset_folder):
    

    direct_comparisons = pd.read_csv(os.path.join(dataset_folder, 'direct_comparisons.csv'))
    # print direct_comparisons
    prev = direct_comparisons[direct_comparisons['actual']==0].shape[0]/float(direct_comparisons.shape[0])
    AUC = metrics.roc_auc_score(direct_comparisons['actual'].values, direct_comparisons['predict'].values)
    return prev, AUC

lab_type = 'component'

all_sites = ['Stanford', 'UCSF', 'UMich']

res_folderpath = 'data-transferring-component-%s/'%curr_version
if not os.path.exists(res_folderpath):
    os.mkdir(res_folderpath)

res_filepath = res_folderpath + 'all_transfers_new.csv'


- Get data

In [29]:
if os.path.exists(res_filepath):
    df_res = pd.read_csv(res_filepath, keep_default_na=False)

else:

    labs = stats_utils.get_important_labs(lab_type='component')


    all_res_dicts = {}
    all_res_dicts['lab'] = labs

    diagonals = []
    off_diags = []

    columns = ['lab', 'prev']
    for i in range(3): # Training sources
        for j in range(3): # Testing sources
            src = all_sites[i]
            dst = all_sites[j]


            '''
            '''
        
            LNL.transfer_labs(src_dataset=src, dst_dataset=dst, lab_type=lab_type,
                                              cur_version=curr_version)
            transfer_result_folderpath = ML_folderpath + '/data-%s-src-%s-dst-%s-%s/' \
                                         % (lab_type, src, dst, curr_version)
            AUCs = []
            Prevs = []
            for lab in labs:
                direct_comparisons_folderpath = os.path.join(transfer_result_folderpath, lab)

                if i!=j:
                    cur_prev, cur_AUC = statistic_analysis(lab=lab, dataset_folder=direct_comparisons_folderpath)
                    off_diags.append(cur_AUC)
                else:
                    tmp_df = pd.read_csv('data-%s-component-10000-episodes-lastnormal' % src
                                    + '/' + 'summary-stats-bestalg-fixTrainPPV.csv', keep_default_na=False)
                    mapped_lab = map_lab(lab=lab, data_source=src, lab_type=lab_type)
                    cur_df = tmp_df[(tmp_df['lab'] == mapped_lab) & (tmp_df['fixTrainPPV'] == 0.95)]
                    cur_AUC = cur_df['AUC'].values[0]
                    cur_prev = (cur_df['TP']+cur_df['FN']).values[0]
                    diagonals.append(cur_AUC)
                AUCs.append(cur_AUC)
                Prevs.append(cur_prev)

            col = '%s -> %s' % (src, dst)
            all_res_dicts[col] = AUCs
            
            all_res_dicts['prev'] = Prevs

            columns.append(col)
    import numpy as np
    print "diagonals avg:", np.mean(diagonals)
    print "off_diags avg:", np.mean(off_diags)

    df_res = pd.DataFrame.from_dict(all_res_dicts)

    descriptions = stats_utils.get_lab_descriptions(lab_type='component')
    df_res['lab'] = df_res['lab'].apply(lambda x:descriptions[x])
    df_res = df_res[columns]
    df_res.to_csv(res_filepath, index=False, float_format='%.2f')

Transfering component WBC from Stanford to Stanford...
Transfering component HGB from Stanford to Stanford...
Transfering component PLT from Stanford to Stanford...
Transfering component NA from Stanford to Stanford...
Transfering component K from Stanford to Stanford...
Transfering component CO2 from Stanford to Stanford...
Transfering component BUN from Stanford to Stanford...
Transfering component CR from Stanford to Stanford...
Transfering component CA from Stanford to Stanford...
Transfering component ALB from Stanford to Stanford...
Transfering component TP from Stanford to Stanford...
Transfering component ALKP from Stanford to Stanford...
Transfering component TBIL from Stanford to Stanford...
Transfering component AST from Stanford to Stanford...
Transfering component ALT from Stanford to Stanford...
Transfering component WBC from Stanford to UCSF...
Transfering component HGB from Stanford to UCSF...
Transfering component PLT from Stanford to UCSF...
Transfering component NA f

### Table 5

In [36]:
cols = df_res.columns.values
def shorten_site(site):
    return site.replace('Stanford','S').replace('UCSF','UC').replace('UMich','UM')
cols_map = dict(zip(cols, (shorten_site(x) for x in cols)))
df_res = df_res.rename(columns=cols_map)
df_res.drop(['prev'], axis=1).to_csv('transfer_table_basic.csv', index=False, float_format='%.2f')

### Figure 5

In [ ]:
# TODO: move this stats part away
import seaborn as sns; sns.set()
import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(16, 12))
col = 5
for ind in range(df_res.shape[0]):
    cur_row = df_res.iloc[ind].values
    print cur_row
    cur_lab = cur_row[0]
    cur_aucs = cur_row[1:].astype(float).reshape(3,3)

    i, j = ind/col, ind%col
    plt.subplot2grid((3, col), (i, j))
    ax = sns.heatmap(cur_aucs, vmin=0, vmax=1, cbar=False, annot=True, cmap='ocean',
                     annot_kws={"size": 18},
                     xticklabels=['S', 'UC', 'UM'], yticklabels=['S', 'UC', 'UM'])
    plt.xlabel(cur_lab, fontsize=20)
    ax.xaxis.set_label_position('top')
    ax.xaxis.set_tick_params(labelsize=18)
    ax.yaxis.set_tick_params(labelsize=18)


plt.tight_layout()
fig.subplots_adjust(hspace=.5)

plt.savefig(res_folderpath + 'transfer_heatmap.png')